### LLM output control

In [6]:
import os
from langchain_google_vertexai import ChatVertexAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# [Optinal] set proxy
proxy = "http://127.0.0.1:7897"
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
os.environ["http_proxy"] = proxy
os.environ["https_proxy"] = proxy

# load google access config file
credential_path="/Users/gongbiao/Code/vertex-ai/config/google_access_token_cp.json"
if os.path.exists(credential_path):
    print(f"the config load success")
else:
    print("config file does'not exists!")
    
# init vertex ai
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

# Use chat mode
model = ChatVertexAI(
    model_name="gemini-1.5-flash",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# Steaming output
#response = model.invoke("Do you know chengdu?")
#print()

the config load success
Yes, I know Chengdu! It's a fascinating city in southwest China, known for:

* **Giant pandas:** Chengdu is home to the Chengdu Research Base of Giant Panda Breeding, a world-renowned facility where you can see these adorable creatures up close.
* **Spicy food:** Chengdu is famous for its Sichuan cuisine, which is known for its bold flavors, especially its use of chili peppers. Some popular dishes include Sichuan hotpot, mapo tofu, and dan dan noodles.
* **Historical sites:** Chengdu has a rich history dating back over 2,000 years. You can visit ancient temples, pagodas, and other historical landmarks.
* **Modern attractions:** Chengdu is a modern city with a vibrant cultural scene, including museums, art galleries, and theaters.
* **Beautiful scenery:** The city is surrounded by mountains and rivers, offering stunning natural beauty.

Do you have any specific questions about Chengdu? I'd be happy to share more information. 



### 自定义输出为 pydantic 对象

In [28]:
import os
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, field_validator



# 定义一个数据模板，用来描述最终的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")
        
    # 验证问题是否符合要求
    @field_validator("setup")
    @classmethod
    def question_mark(cls, field):
        if field[-1] not in ["?", "？"]:
            raise ValueError("不符合预期的问题格式!")
        return field
    

# 设置 Joke 数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

# 设置提示词模板
prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions": parser.get_format_instructions()}
)

# ---------------------------------

# [Optinal] set proxy
proxy = "http://127.0.0.1:8889"
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
os.environ["http_proxy"] = proxy
os.environ["https_proxy"] = proxy

# load google access config file
credential_path="/Users/gongbiao/Code/vertex-ai/config/google_access_token_cp.json"
if os.path.exists(credential_path):
    print(f"the config load success")
else:
    print("config file does'not exists!")
    
# init vertex ai
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
# Use chat mode
model = ChatVertexAI(model_name="gemini-1.5-flash", temperature=0)

prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "给老子讲个笑话"})
print("output:", output)
parser.invoke(output)




the config load success
output: content='```json\n{"setup": "为什么企鹅不会飞？", "punchline": "因为他们太胖了，而且穿了燕尾服！"}\n```' additional_kwargs={} response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'LOW', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'LOW', 'blocked': False, 'severity': 'HARM_SEVERITY_LOW'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 197, 'candidates_token_count': 35, 'total_token_count': 232, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.12626259667532785} id='run-262ba6c3-3e6d-4add-a2e8-309f8ec27dc0-0' usage_metadata={'input_tokens': 1

Joke(setup='为什么企鹅不会飞？', punchline='因为他们太胖了，而且穿了燕尾服！')

In [47]:

### LLM 输出格式化成 python list 形式，类似['a', 'b', 'c']
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
import os

parser = CommaSeparatedListOutputParser()
prompt = PromptTemplate(
    template="列出{count}个{subject}.\n{format_instructions}",
    input_variables=["count", "subject"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format(count="3", subject="常见的中国田园犬名字")
# Create a HumanMessage object with the input
human_message = HumanMessage(content=_input)


# ---------------------------------

# [Optinal] set proxy
proxy = "http://127.0.0.1:8889"
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
os.environ["http_proxy"] = proxy
os.environ["https_proxy"] = proxy

# load google access config file
credential_path="/Users/gongbiao/Code/vertex-ai/config/google_access_token_cp.json"
if os.path.exists(credential_path):
    print(f"the config load success")
else:
    print("config file does'not exists!")
    
# init vertex ai
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
# Use chat mode
model = ChatVertexAI(model_name="gemini-1.5-flash", temperature=0)

response = model([human_message])
print(response.content)

# Format
output = parser.parse(response.content)
print("type:", type(output))
print(output)


the config load success
土狗, 大黄, 花花 

type: <class 'list'>
['土狗', '大黄', '花花']
